<a href="https://colab.research.google.com/github/piltom/widget_sketch_recon/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimiento de widgets a mano alzada para generar UIs a partir de un borrador
## Big Picture
Este trabajo es una parte de un proyecto más grande, en el cual se intenta generar una descripción de una interfaz de usuario a partir de un sketch en papel/pizarra. Con esta descripción se puede después generar código para algún lenguaje/framwork en particular (ReactJS, Qt, Gtk, etc.)
Un ejemplo de un par de sketch e interfaz es el siguiente:


